In [51]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum as sp_sum

In [52]:
# Jupyter 노트북 셀에서 PySpark 코드 실행
from pyspark.sql import SparkSession


# Spark 세션 생성
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("PySparkTest") \
    .getOrCreate()

In [53]:
# meta_sample.csv 경로
csv_path = '/Users/b06/Desktop/yeardream/medi-05/data/meta_sample.csv'


# 데이터 로드
df = spark.read.csv(csv_path, header=True)

In [55]:
# count Nan
nan_cnt = df.select([sp_sum(col(c).isNull().cast("int")).alias(c) for c in df.columns])


# df에 nan_cnt 열 추가
df_nan_cnt = df.withColumn('nan_count', sum(col(c).isNull().cast("int") for c in df.columns))


# nan_cnt 값이 가장 적은 row
min_nan_row = df_nan_cnt.orderBy("nan_count").first()


# min_nan_row 출력
for k, v in min_nan_row.asDict().items():
    print(f"{k}: {v}")

id: 1070175780
name: 답십리경희한의원
review_settings_keyword: (답십리경희한의원) & (동대문구 | 답십리동 | 한의원-일반) & 답십리경희한의원
keywords_1: 답십리한의원
keywords_2: 전농동한의원
keywords_3: 청량리한의원
keywords_4: 답십리야간진료한의원
keywords_5: 답십리교통사고한의원
menu_name: None
menu_price: None
menu_description: None
menu_images: None
visitor_reviews_total: 1107
photo_review_ratio: 0.044263775971093
fsas_reviews_total: 383
images_count: 25
naver_booking_url: https://m.booking.naver.com/booking/13/bizes/536100
talktalk_url: None
road: -래미안 미드카운티 아파트 맞은편 카운티에비뉴 3층-건물 내 무료주차 가능합니다.
virtual_phone: 0507-1482-7946
phone: None
is_smart_phone: True
is_blog_exposed: True
zeropay_available: False
conveniences_1: 단체 이용 가능
conveniences_2: 주차
conveniences_3: 예약
conveniences_4: 무선 인터넷
conveniences_5: 남/녀 화장실 구분
conveniences_6: 장애인 편의시설
conveniences_7: None
conveniences_8: None
conveniences_9: None
conveniences_10: None
conveniences_11: None
homepages_url: https://blog.naver.com/dapsimni_kmc
homepages_landingUrl: https://blog.naver.com/dapsimni_kmc
homepage

In [56]:
# Select specific columns
selected_columns = [
    'id', 'name', 'keywords_1', 'keywords_2', 'keywords_3', 
    'keywords_4', 'keywords_5', 'visitor_reviews_total', 
    'photo_review_ratio', 'fsas_reviews_total', 'images_count', 
    'talktalk_url', 'homepages_type', 'homepages_isDeadUrl', 
    'homepages_isRep', 'description_length', 'self_blog_present', 
    'qna_answer_count', 'crawled_at'
]

# Create a new DataFrame with only the selected columns
new_df = df.select(selected_columns)

# Show the first few rows of the new DataFrame
new_df.show()

+----------+-----------------+------------+--------------+--------------+----------------+------------+---------------------+------------------+------------------+------------+--------------------+--------------+-------------------+---------------+------------------+-----------------+----------------+--------------------+
|        id|             name|  keywords_1|    keywords_2|    keywords_3|      keywords_4|  keywords_5|visitor_reviews_total|photo_review_ratio|fsas_reviews_total|images_count|        talktalk_url|homepages_type|homepages_isDeadUrl|homepages_isRep|description_length|self_blog_present|qna_answer_count|          crawled_at|
+----------+-----------------+------------+--------------+--------------+----------------+------------+---------------------+------------------+------------------+------------+--------------------+--------------+-------------------+---------------+------------------+-----------------+----------------+--------------------+
|  19518309|       송정한의원|   

In [ ]:
# Assuming new_df is the DataFrame you want to save
new_df.write.csv('/Users/b06/Desktop/yeardream/medi-05/data/new_dataset.csv', header=True)